In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os

from jamie import JAMIE
from jamie.evaluation import *
from jamie.utilities import *
import matplotlib.pyplot as plt
from mmd_wrapper import mmd_combine
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import preprocessing

2023-01-12 23:39:14.395774: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-12 23:39:14.475566: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-12 23:39:14.478720: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-12 23:39:14.478729: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
np.random.seed(42)

In [4]:
kwargs = {
    'output_dim': 32,
    'epoch_DNN': 10000,
    'min_epochs': 2500,
    'log_DNN': 500,
    'use_early_stop': True,
    'batch_size': 512,
    'pca_dim': 2*[512],
    'dist_method': 'euclidean',
    'loss_weights': [1,1,1,1],
}

# MMD-MA

In [5]:
dataset_name = 'MMD-MA'
dataset_color = 'lime'
modality_names = ['Modality 1', 'Modality 2']
data_folder = '../data/UnionCom/MMD/'
data1 = np.loadtxt(data_folder + "s1_mapped1.txt")
data2 = np.loadtxt(data_folder + "s1_mapped2.txt")
type1 = np.loadtxt(data_folder + "s1_type1.txt")
type2 = np.loadtxt(data_folder + "s1_type2.txt")
type1 = type1.astype(np.int)
type2 = type2.astype(np.int)
type1 = np.array([f'Cell Type {i}' for i in type1])
type2 = np.array([f'Cell Type {i}' for i in type2])

# Labels
labels = [type1, type2]
features = [None, None]
feature_dict = {}

/tmp/ipykernel_1272/4179390009.py:9: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  type1 = type1.astype(np.int)
/tmp/ipykernel_1272/4179390009.py:10: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for mo

In [6]:
# Preprocessing
data1 = preprocessing.scale(data1, axis=0)
data2 = preprocessing.scale(data2, axis=0)
data1[np.isnan(data1)] = 0  # Replace NaN with average
data2[np.isnan(data2)] = 0
# data1 = preprocessing.MinMaxScaler().fit_transform(data1)
# data2 = preprocessing.MinMaxScaler().fit_transform(data2)
dataset = [data1, data2]

# Replace NULL feature names
for i in range(len(features)):
    if features[i] is None:
        features[i] = np.array([f'Feature {i}' for i in range(dataset[i].shape[1])])
        
# Train-Test Imputation
train_size = int(.8 * len(data1))
train_idx = np.random.choice(range(len(data1)), train_size, replace=False)
test_idx = np.array(list(set(range(len(data1))) - set(train_idx)))

# Reduced Priors
full_priors = np.eye(len(dataset[0]))
random_idx = np.random.choice(range(len(dataset[0])), int(.5 * len(dataset[0])), replace=False)
priors = np.zeros(len(dataset[0]))
priors[random_idx] = 1
half_priors = np.diag(priors)
random_idx = np.random.choice(range(len(dataset[0])), int(.75 * len(dataset[0])), replace=False)
priors = np.zeros(len(dataset[0]))
priors[random_idx] = 1
tq_priors = np.diag(priors)
none_priors = np.zeros((len(dataset[0]), len(dataset[0])))

In [7]:
jm = JAMIE(**kwargs, debug=True, enable_memory_logging=True)
jm_data = jm.fit_transform(dataset=dataset)

use random seed: 666
Shape of Raw data
Dataset 0: (300, 2000)
Dataset 1: (300, 1000)
Device: cpu
---------------------------------
Find correspondence between Dataset 1 and Dataset 2
epoch:[500/2000] err:11.7152 alpha:0.1578
epoch:[1000/2000] err:10.2456 alpha:0.3746
epoch:[1500/2000] err:10.1076 alpha:0.4565
epoch:[2000/2000] err:10.0670 alpha:0.4759
Finished Matching!
---------------------------------
Train coupled autoencoders


/mnt/c/Users/nck/repos/nmacom/jamie/jamie.py:427: UserWarning: PCA dim must be lower than 300, found 512, adjusting to compensate.
  warnings.warn(
/mnt/c/Users/nck/repos/nmacom/jamie/jamie.py:427: UserWarning: PCA dim must be lower than 300, found 512, adjusting to compensate.
  warnings.warn(


Epoch: 100 - KL: 0.0016  Rec: 0.3937  CosSim: 0.1242  F: 0.0076
Epoch: 200 - KL: 0.0034  Rec: 0.3471  CosSim: 0.0354  F: 0.0052
Epoch: 300 - KL: 0.0075  Rec: 0.3299  CosSim: 0.0275  F: 0.0049
Epoch: 400 - KL: 0.0104  Rec: 0.3238  CosSim: 0.0215  F: 0.0036
Epoch: 500 - KL: 0.0132  Rec: 0.3179  CosSim: 0.0218  F: 0.0042
epoch:[500/10000]: loss:0.357045
Epoch: 600 - KL: 0.0167  Rec: 0.3220  CosSim: 0.0197  F: 0.0039
Epoch: 700 - KL: 0.0255  Rec: 0.3097  CosSim: 0.0219  F: 0.0041
Epoch: 800 - KL: 0.0416  Rec: 0.3024  CosSim: 0.0233  F: 0.0036
Epoch: 900 - KL: 0.0559  Rec: 0.3068  CosSim: 0.0232  F: 0.0038
Epoch: 1000 - KL: 0.0682  Rec: 0.3027  CosSim: 0.0244  F: 0.0036
epoch:[1000/10000]: loss:0.399021
Epoch: 1100 - KL: 0.0855  Rec: 0.2954  CosSim: 0.0286  F: 0.0034
Epoch: 1200 - KL: 0.0886  Rec: 0.2994  CosSim: 0.0341  F: 0.0040
Epoch: 1300 - KL: 0.1167  Rec: 0.2974  CosSim: 0.0370  F: 0.0034
Epoch: 1400 - KL: 0.1138  Rec: 0.2984  CosSim: 0.0402  F: 0.0039
Epoch: 1500 - KL: 0.1841  Rec: 0

# Simulation 1250

In [8]:
dataset_name = 'scMultiSim-1250'
dataset_color = 'teal'
modality_names = ['Modality 1', 'Modality 2']
data_folder = '../data/scMultiSim/new/'
data1 = np.loadtxt(data_folder + "scMultiSim_RNA_counts_1250_genes.csv", delimiter=",", skiprows=1)
data2 = np.loadtxt(data_folder + "scMultiSim_ATAC_seq_1250_genes_new.csv", delimiter=",", skiprows=1)
ct = pd.read_csv(data_folder + "cell_meta_1250_genes.csv").iloc[3, 1:].to_numpy().astype(int)
type1 = type2 = np.array([f'Cell Type {i}' for i in ct])

# Labels
labels = [type1, type2]
features = [None, None]
feature_dict = {}

# Utility
positivize = lambda X: [x + x.min() for x in X]
minmax = lambda X: [(x + x.min()) for x in X]

In [9]:
# Preprocessing
data1 = preprocessing.scale(data1, axis=0)
data2 = preprocessing.scale(data2, axis=0)
data1[np.isnan(data1)] = 0  # Replace NaN with average
data2[np.isnan(data2)] = 0
# data1 = preprocessing.MinMaxScaler().fit_transform(data1)
# data2 = preprocessing.MinMaxScaler().fit_transform(data2)
dataset = [data1, data2]

# Replace NULL feature names
for i in range(len(features)):
    if features[i] is None:
        features[i] = np.array([f'Feature {i}' for i in range(dataset[i].shape[1])])
        
# Train-Test Imputation
train_size = int(.8 * len(data1))
train_idx = np.random.choice(range(len(data1)), train_size, replace=False)
test_idx = np.array(list(set(range(len(data1))) - set(train_idx)))

# Reduced Priors
full_priors = np.eye(len(dataset[0]))
random_idx = np.random.choice(range(len(dataset[0])), int(.5 * len(dataset[0])), replace=False)
priors = np.zeros(len(dataset[0]))
priors[random_idx] = 1
half_priors = np.diag(priors)
random_idx = np.random.choice(range(len(dataset[0])), int(.75 * len(dataset[0])), replace=False)
priors = np.zeros(len(dataset[0]))
priors[random_idx] = 1
tq_priors = np.diag(priors)
none_priors = np.zeros((len(dataset[0]), len(dataset[0])))

In [10]:
jm = JAMIE(**kwargs, debug=True, enable_memory_logging=True)
jm_data = jm.fit_transform(dataset=dataset)

use random seed: 666
Shape of Raw data
Dataset 0: (500, 1250)
Dataset 1: (500, 3750)
Device: cpu
---------------------------------
Find correspondence between Dataset 1 and Dataset 2
epoch:[500/2000] err:13.0043 alpha:0.0774
epoch:[1000/2000] err:16.3444 alpha:0.1053
epoch:[1500/2000] err:18.7388 alpha:0.1709
epoch:[2000/2000] err:21.0007 alpha:0.4030
Finished Matching!
---------------------------------
Train coupled autoencoders


/mnt/c/Users/nck/repos/nmacom/jamie/jamie.py:427: UserWarning: PCA dim must be lower than 500, found 512, adjusting to compensate.
  warnings.warn(
/mnt/c/Users/nck/repos/nmacom/jamie/jamie.py:427: UserWarning: PCA dim must be lower than 500, found 512, adjusting to compensate.
  warnings.warn(


Epoch: 100 - KL: 0.0025  Rec: 1.1084  CosSim: 0.0669  F: 0.0054
Epoch: 200 - KL: 0.0040  Rec: 0.9321  CosSim: 0.0375  F: 0.0052
Epoch: 300 - KL: 0.0064  Rec: 0.8628  CosSim: 0.0319  F: 0.0054
Epoch: 400 - KL: 0.0090  Rec: 0.8166  CosSim: 0.0298  F: 0.0053
Epoch: 500 - KL: 0.0143  Rec: 0.7929  CosSim: 0.0271  F: 0.0052
epoch:[500/10000]: loss:0.839573
Epoch: 600 - KL: 0.0170  Rec: 0.7661  CosSim: 0.0248  F: 0.0046
Epoch: 700 - KL: 0.0279  Rec: 0.7453  CosSim: 0.0251  F: 0.0044
Epoch: 800 - KL: 0.0437  Rec: 0.7315  CosSim: 0.0250  F: 0.0047
Epoch: 900 - KL: 0.0384  Rec: 0.7200  CosSim: 0.0256  F: 0.0044
Epoch: 1000 - KL: 0.0732  Rec: 0.7135  CosSim: 0.0264  F: 0.0044
epoch:[1000/10000]: loss:0.817421
Epoch: 1100 - KL: 0.0787  Rec: 0.7092  CosSim: 0.0291  F: 0.0044
Epoch: 1200 - KL: 0.1102  Rec: 0.7003  CosSim: 0.0314  F: 0.0045
Epoch: 1300 - KL: 0.1421  Rec: 0.6863  CosSim: 0.0354  F: 0.0047
Epoch: 1400 - KL: 0.1302  Rec: 0.6853  CosSim: 0.0388  F: 0.0047
Epoch: 1500 - KL: 0.1455  Rec: 0

# scMNC Motor

In [11]:
dataset_name = 'scMNC-Motor'
dataset_color = 'sienna'
modality_names = ['Gene Expression', 'Electrophysiology']
data_folder = '../data/scMNC/mouse_motor_cortex/data/'
data1 = pd.read_csv(data_folder + "geneExp_filtered.csv")
data2 = pd.read_csv(data_folder + "efeature_filtered.csv")
feat1 = np.array(data1.iloc[:, 0])
feat2 = np.array(data2.columns)
sample_names1 = data1.columns[1:]
assert ((data1.shape[1] - 1) == data2.shape[0])
data1 = np.transpose(np.array(data1)[:, 1:])
data2 = np.array(data2)
meta = pd.read_excel(data_folder + "motor_meta_data.xlsx")[['Cell', 'RNA family']]
meta = np.array(meta)
meta_idx = [np.argwhere(meta[:, 0] == sample_names1[i])[0][0] for i in range(sample_names1.shape[0])]
type1 = type2 = np.array([x.split()[0] for x in meta[meta_idx, 1]])

# Sampling
# split = 1000 # data1.shape[0]
# data_col_idx = np.random.choice(range(data1.shape[0]), split, replace=False)
# data1, data2, type1, type2 = (x[data_col_idx] for x in (data1, data2, type1, type2))

# Labels
labels = [type1, type2]
features = [feat1, feat2]
feature_dict = {}

# Utility
positivize = lambda X: [x + x.min() for x in X]
minmax = lambda X: [(x + x.min()) for x in X]

In [12]:
# Preprocessing
data1 = preprocessing.scale(data1, axis=0)
data2 = preprocessing.scale(data2, axis=0)
data1[np.isnan(data1)] = 0  # Replace NaN with average
data2[np.isnan(data2)] = 0
# data1 = preprocessing.MinMaxScaler().fit_transform(data1)
# data2 = preprocessing.MinMaxScaler().fit_transform(data2)
dataset = [data1, data2]

# Replace NULL feature names
for i in range(len(features)):
    if features[i] is None:
        features[i] = np.array([f'Feature {i}' for i in range(dataset[i].shape[1])])
        
# Train-Test Imputation
train_size = int(.8 * len(data1))
train_idx = np.random.choice(range(len(data1)), train_size, replace=False)
test_idx = np.array(list(set(range(len(data1))) - set(train_idx)))

# Reduced Priors
full_priors = np.eye(len(dataset[0]))
random_idx = np.random.choice(range(len(dataset[0])), int(.5 * len(dataset[0])), replace=False)
priors = np.zeros(len(dataset[0]))
priors[random_idx] = 1
half_priors = np.diag(priors)
random_idx = np.random.choice(range(len(dataset[0])), int(.75 * len(dataset[0])), replace=False)
priors = np.zeros(len(dataset[0]))
priors[random_idx] = 1
tq_priors = np.diag(priors)
none_priors = np.zeros((len(dataset[0]), len(dataset[0])))

In [13]:
jm = JAMIE(**kwargs, debug=True, enable_memory_logging=True)
jm_data = jm.fit_transform(dataset=dataset)

use random seed: 666
Shape of Raw data
Dataset 0: (1208, 1286)
Dataset 1: (1208, 29)
Device: cpu
---------------------------------
Find correspondence between Dataset 1 and Dataset 2
epoch:[500/2000] err:0.3861 alpha:0.0050
epoch:[1000/2000] err:3.3359 alpha:0.0249
epoch:[1500/2000] err:5.7469 alpha:0.0598
epoch:[2000/2000] err:7.0837 alpha:0.0926
Finished Matching!
---------------------------------
Train coupled autoencoders


/mnt/c/Users/nck/repos/nmacom/jamie/jamie.py:427: UserWarning: PCA dim must be lower than 29, found 512, adjusting to compensate.
  warnings.warn(


Epoch: 100 - KL: 0.0007  Rec: 1.7173  CosSim: 1.3189  F: 0.0223
Epoch: 200 - KL: 0.0029  Rec: 1.6833  CosSim: 0.3699  F: 0.0063
Epoch: 300 - KL: 0.0045  Rec: 1.4586  CosSim: 0.1846  F: 0.0035
Epoch: 400 - KL: 0.0076  Rec: 1.4519  CosSim: 0.1341  F: 0.0028
Epoch: 500 - KL: 0.0099  Rec: 1.4924  CosSim: 0.0905  F: 0.0023
epoch:[500/10000]: loss:1.512238
Epoch: 600 - KL: 0.0183  Rec: 1.3405  CosSim: 0.0826  F: 0.0020
Epoch: 700 - KL: 0.0260  Rec: 1.3499  CosSim: 0.0944  F: 0.0020
Epoch: 800 - KL: 0.0441  Rec: 1.3492  CosSim: 0.0684  F: 0.0017
Epoch: 900 - KL: 0.0733  Rec: 1.3068  CosSim: 0.0543  F: 0.0016
Epoch: 1000 - KL: 0.0943  Rec: 1.1985  CosSim: 0.0617  F: 0.0016
epoch:[1000/10000]: loss:1.345666
Epoch: 1100 - KL: 0.1174  Rec: 1.2670  CosSim: 0.0648  F: 0.0018
Epoch: 1200 - KL: 0.1444  Rec: 1.2078  CosSim: 0.0632  F: 0.0016
Epoch: 1300 - KL: 0.2010  Rec: 1.1388  CosSim: 0.0661  F: 0.0016
Epoch: 1400 - KL: 0.1848  Rec: 1.1798  CosSim: 0.0663  F: 0.0016
Epoch: 1500 - KL: 0.1324  Rec: 1

# scMNC Visual

In [14]:
dataset_name = 'scMNC-Visual'
dataset_color = 'magenta'
modality_names = ['Gene Expression', 'Electrophysiology']
data_folder = '../data/scMNC/mouse_visual_cortex/data/'
data1 = pd.read_csv(data_folder + "geneExp_filtered.csv")
data2 = pd.read_csv(data_folder + "efeature_filtered.csv")
sample_names1 = data1.columns[1:]
sample_names2 = np.array(data2)[:, 0]
feature_names1 = data1.iloc[:,0]
feature_names2 = data2.columns[3:]
assert (sample_names1 == sample_names2).all()
data1 = np.transpose(np.array(data1)[:, 1:])
data2 = np.array(data2)[:, 3:]
meta = pd.read_csv(data_folder + "20200711_patchseq_metadata_mouse.csv")[['transcriptomics_sample_id', 't_type']]
meta = np.array(meta)
meta_idx = [np.argwhere(meta[:, 0] == sample_names1[i])[0][0] for i in range(sample_names1.shape[0])]
type1 = type2 = np.array([x.split(' ')[0] for x in meta[meta_idx, 1]])

# Sampling
# split = 1000 # data1.shape[0]
# data_col_idx = np.random.choice(range(data1.shape[0]), split, replace=False)
# data1, data2, type1, type2 = (x[data_col_idx] for x in (data1, data2, type1, type2))

# Labels
labels = [type1, type2]
features = [np.array(feature_names1), np.array(feature_names2)]
feature_dict = {}

# Utility
positivize = lambda X: [x + x.min() for x in X]
minmax = lambda X: [(x + x.min()) for x in X]

In [15]:
# Preprocessing
data1 = preprocessing.scale(data1, axis=0)
data2 = preprocessing.scale(data2, axis=0)
data1[np.isnan(data1)] = 0  # Replace NaN with average
data2[np.isnan(data2)] = 0
# data1 = preprocessing.MinMaxScaler().fit_transform(data1)
# data2 = preprocessing.MinMaxScaler().fit_transform(data2)
dataset = [data1, data2]

# Replace NULL feature names
for i in range(len(features)):
    if features[i] is None:
        features[i] = np.array([f'Feature {i}' for i in range(dataset[i].shape[1])])
        
# Train-Test Imputation
train_size = int(.8 * len(data1))
train_idx = np.random.choice(range(len(data1)), train_size, replace=False)
test_idx = np.array(list(set(range(len(data1))) - set(train_idx)))

# Reduced Priors
full_priors = np.eye(len(dataset[0]))
random_idx = np.random.choice(range(len(dataset[0])), int(.5 * len(dataset[0])), replace=False)
priors = np.zeros(len(dataset[0]))
priors[random_idx] = 1
half_priors = np.diag(priors)
random_idx = np.random.choice(range(len(dataset[0])), int(.75 * len(dataset[0])), replace=False)
priors = np.zeros(len(dataset[0]))
priors[random_idx] = 1
tq_priors = np.diag(priors)
none_priors = np.zeros((len(dataset[0]), len(dataset[0])))

In [16]:
jm = JAMIE(**kwargs, debug=True, enable_memory_logging=True)
jm_data = jm.fit_transform(dataset=dataset)

use random seed: 666
Shape of Raw data
Dataset 0: (3654, 1302)
Dataset 1: (3654, 39)
Device: cpu
---------------------------------
Find correspondence between Dataset 1 and Dataset 2
epoch:[500/2000] err:0.4526 alpha:0.0207
epoch:[1000/2000] err:0.0000 alpha:0.0000
epoch:[1500/2000] err:0.0000 alpha:0.0000
epoch:[2000/2000] err:0.0000 alpha:0.0000
Finished Matching!
---------------------------------
Train coupled autoencoders


/mnt/c/Users/nck/repos/nmacom/jamie/jamie.py:427: UserWarning: PCA dim must be lower than 39, found 512, adjusting to compensate.
  warnings.warn(


Epoch: 100 - KL: 0.0027  Rec: 1.4028  CosSim: 0.0921  F: 0.0023
Epoch: 200 - KL: 0.0040  Rec: 1.2102  CosSim: 0.0425  F: 0.0015
Epoch: 300 - KL: 0.0061  Rec: 1.1673  CosSim: 0.0318  F: 0.0012
Epoch: 400 - KL: 0.0128  Rec: 1.1311  CosSim: 0.0284  F: 0.0009
Epoch: 500 - KL: 0.0231  Rec: 1.2293  CosSim: 0.0237  F: 0.0009
epoch:[500/10000]: loss:1.293293
Epoch: 600 - KL: 0.0261  Rec: 1.3639  CosSim: 0.0233  F: 0.0009
Epoch: 700 - KL: 0.0365  Rec: 1.2801  CosSim: 0.0328  F: 0.0010
Epoch: 800 - KL: 0.0487  Rec: 1.1628  CosSim: 0.0269  F: 0.0008
Epoch: 900 - KL: 0.0622  Rec: 1.1354  CosSim: 0.0258  F: 0.0008
Epoch: 1000 - KL: 0.0842  Rec: 1.1176  CosSim: 0.0255  F: 0.0007
epoch:[1000/10000]: loss:1.209488
Epoch: 1100 - KL: 0.0747  Rec: 1.0409  CosSim: 0.0271  F: 0.0007
Epoch: 1200 - KL: 0.1079  Rec: 1.0813  CosSim: 0.0291  F: 0.0007
Epoch: 1300 - KL: 0.1372  Rec: 1.0874  CosSim: 0.0340  F: 0.0007
Epoch: 1400 - KL: 0.1532  Rec: 1.1358  CosSim: 0.0403  F: 0.0007
Epoch: 1500 - KL: 0.1515  Rec: 1

# DM_rep4

In [17]:
import scanpy as sc

babel_dir = '../data/babel_data/DM_rep4/'
train = sc.read_h5ad(babel_dir + 'train_rna.h5ad')
v = train.var_names
train = train.X.toarray()
valid = sc.read_h5ad(babel_dir + 'truth_rna.h5ad').X.toarray()
data1 = np.concatenate([train, valid], axis=0)
fnames1 = np.array(v)

train = sc.read_h5ad(babel_dir + 'train_atac.h5ad')
v = train.var_names
train = train.X.toarray()
valid = sc.read_h5ad(babel_dir + 'truth_atac.h5ad').X.toarray()
data2 = np.concatenate([train, valid], axis=0)
fnames2 = np.array(v)

split = train.shape[0]

/home/thema/miniconda3/lib/python3.9/site-packages/anndata/compat/__init__.py:232: FutureWarning: Moving element from .uns['neighbors']['distances'] to .obsp['distances'].

This is where adjacency matrices should go now.
  warn(
/home/thema/miniconda3/lib/python3.9/site-packages/anndata/compat/__init__.py:232: FutureWarning: Moving element from .uns['neighbors']['connectivities'] to .obsp['connectivities'].

This is where adjacency matrices should go now.
  warn(
/home/thema/miniconda3/lib/python3.9/site-packages/anndata/compat/__init__.py:232: FutureWarning: Moving element from .uns['neighbors']['distances'] to .obsp['distances'].

This is where adjacency matrices should go now.
  warn(
/home/thema/miniconda3/lib/python3.9/site-packages/anndata/compat/__init__.py:232: FutureWarning: Moving element from .uns['neighbors']['connectivities'] to .obsp['connectivities'].

This is where adjacency matrices should go now.
  warn(


In [18]:
dataset_name = 'DM_rep4'
dataset_color = 'black'
modality_names = ['RNA', 'ATAC']

# data1 = PCA(n_components=32).fit_transform(data1)
# data2 = PCA(n_components=32).fit_transform(data2)
# fnames1 = fnames2 = None

type1 = np.array(len(data1) * ['Cell Type 0'])
type2 = np.array(len(data2) * ['Cell Type 0'])

# Sampling
# sample_num = 500
# data_col_idx = np.random.choice(range(split), sample_num, replace=False)
# data1, data2, type1, type2 = (x[list(data_col_idx) + list(range(split, len(data1)))] for x in (data1, data2, type1, type2))
# split = sample_num

# Labels
labels = [type1, type2]
features = [np.array(fnames1), np.array(fnames2)]
feature_dict = {}

# Utility
positivize = lambda X: [x + x.min() for x in X]
minmax = lambda X: [(x + x.min()) for x in X]

In [19]:
# Preprocessing
data1 = preprocessing.scale(data1, axis=0)
data2 = preprocessing.scale(data2, axis=0)
data1[np.isnan(data1)] = 0  # Replace NaN with average
data2[np.isnan(data2)] = 0
# data1 = preprocessing.MinMaxScaler().fit_transform(data1)
# data2 = preprocessing.MinMaxScaler().fit_transform(data2)
dataset = [data1, data2]

# Replace NULL feature names
for i in range(len(features)):
    if features[i] is None:
        features[i] = np.array([f'Feature {i}' for i in range(dataset[i].shape[1])])
        
# # Train-Test Imputation
train_size = split
train_idx = np.array(range(split))
test_idx = np.array(list(set(range(len(data1))) - set(train_idx)))

# Reduced Priors
full_priors = np.eye(len(dataset[0]))
random_idx = np.random.choice(range(len(dataset[0])), int(.5 * len(dataset[0])), replace=False)
priors = np.zeros(len(dataset[0]))
priors[random_idx] = 1
half_priors = np.diag(priors)
none_priors = np.zeros((len(dataset[0]), len(dataset[0])))

/home/thema/miniconda3/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:235: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
/home/thema/miniconda3/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:254: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(
/home/thema/miniconda3/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:235: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
/home/thema/miniconda3/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:254: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. 

In [20]:
jm = JAMIE(**kwargs, debug=True, enable_memory_logging=True)
jm_data = jm.fit_transform(dataset=dataset)

use random seed: 666
Shape of Raw data
Dataset 0: (4301, 34861)
Dataset 1: (4301, 85596)
Device: cpu
---------------------------------
Find correspondence between Dataset 1 and Dataset 2
epoch:[500/2000] err:3240.7747 alpha:318.6570
epoch:[1000/2000] err:202.1322 alpha:18.4804
epoch:[1500/2000] err:0.6152 alpha:0.0076
epoch:[2000/2000] err:0.0001 alpha:0.0000
Finished Matching!
---------------------------------
Train coupled autoencoders
Epoch: 100 - KL: 0.0026  Rec: 1.8253  CosSim: 0.0447  F: 0.0020
Epoch: 200 - KL: 0.0046  Rec: 1.8538  CosSim: 0.0469  F: 0.0033
Epoch: 300 - KL: 0.0065  Rec: 1.6934  CosSim: 0.0374  F: 0.0035
Epoch: 400 - KL: 0.0089  Rec: 1.4560  CosSim: 0.0336  F: 0.0026
Epoch: 500 - KL: 0.0147  Rec: 1.5693  CosSim: 0.0241  F: 0.0016
epoch:[500/10000]: loss:1.573188
Epoch: 600 - KL: 0.0205  Rec: 1.6335  CosSim: 0.0203  F: 0.0018
Epoch: 700 - KL: 0.0304  Rec: 1.5816  CosSim: 0.0277  F: 0.0036
Epoch: 800 - KL: 0.0393  Rec: 1.4780  CosSim: 0.0256  F: 0.0015
Epoch: 900 - 

# brainchromatin

In [21]:
dataset_name = 'BrainChromatin'
dataset_color = 'orange'
modality_names = ['RNA', 'ATAC']
data_folder = '../data/brainchromatin/'
data1 = pd.read_csv(data_folder + "multiome_rna_counts.tsv", delimiter='\t').transpose()
data2 = pd.read_csv(data_folder + "multiome_atac_gene_activities.tsv", delimiter='\t', nrows=35000).transpose()
data2 = data2.transpose()[data1.index].transpose()

meta = pd.read_csv(data_folder + "multiome_cell_metadata.txt", delimiter='\t')
meta_names = pd.read_csv(data_folder + "multiome_cluster_names.txt", delimiter='\t')
meta_names = meta_names[meta_names['Assay'] == 'Multiome ATAC']
meta = pd.merge(meta, meta_names, left_on='ATAC_cluster', right_on='Cluster.ID', how='left')
meta.index = meta['Cell.ID']

type1 = type2 = np.array(meta.transpose()[data1.index].transpose()['Cluster.Name'])
fname1, fname2 = data1.columns, data2.columns
data1 = data1.to_numpy()
data2 = data2.to_numpy()

# Sampling
split = 4000 # data1.shape[0]
data_row_idx = np.random.choice(range(data1.shape[0]), split, replace=False)
# data1, data2, type1, type2 = (x[data_row_idx] for x in (data1, data2, type1, type2))
# split_feat_1 = 2000 # data1.shape[1]
# data_col1_idx = np.random.choice(range(data1.shape[1]), split_feat_1, replace=False)
# data1, fname1 = data1[:, data_col1_idx], fname1[data_col1_idx]
# split_feat_2 = 2000 # data2.shape[1]
# data_col2_idx = np.random.choice(range(data2.shape[1]), split_feat_2, replace=False)
# data2, fname2 = data2[:, data_col2_idx], fname2[data_col2_idx]

# Labels
labels = [type1, type2]
features = [np.array(fname1), np.array(fname2)]
feature_dict = {'ENSG00000251562': 'MALAT1', 'ENSG00000153707': 'PTPRD'}

# Utility
positivize = lambda X: [x + x.min() for x in X]
minmax = lambda X: [(x + x.min()) for x in X]

# Change Labels
group = ['GluN3', 'GluN4', 'IN1', 'GluN2', 'IN2', 'GluN6', 'GluN5', 'RG',
       'nIPC', 'GluN1', 'mGPC/OPC', 'IN3', 'IN4', 'SP', 'GluN7',
       'MG/EC/Peric.']
conv =  ['GluN', 'GluN', 'IN', 'GluN', 'IN', 'GluN', 'GluN', 'RG',
        'nIPC', 'GluN', 'mGPC/OPC', 'IN', 'IN', 'SP', 'GluN',
        'MG/EC/Peric.']
group_conv = {g:c for g, c in zip(group, conv)}
labels = [np.array([group_conv[l] for l in label]) for label in labels]

In [22]:
# Preprocessing
data1 = preprocessing.scale(data1, axis=0)
data2 = preprocessing.scale(data2, axis=0)
data1[np.isnan(data1)] = 0  # Replace NaN with average
data2[np.isnan(data2)] = 0
# data1 = preprocessing.MinMaxScaler().fit_transform(data1)
# data2 = preprocessing.MinMaxScaler().fit_transform(data2)
dataset = [data1, data2]

# Replace NULL feature names
for i in range(len(features)):
    if features[i] is None:
        features[i] = np.array([f'Feature {i}' for i in range(dataset[i].shape[1])])
        
# Train-Test Imputation
train_size = int(.8 * len(data1))
train_idx = np.random.choice(range(len(data1)), train_size, replace=False)
test_idx = np.array(list(set(range(len(data1))) - set(train_idx)))

# Reduced Priors
full_priors = np.eye(len(dataset[0]))
random_idx = np.random.choice(range(len(dataset[0])), int(.5 * len(dataset[0])), replace=False)
priors = np.zeros(len(dataset[0]))
priors[random_idx] = 1
half_priors = np.diag(priors)
random_idx = np.random.choice(range(len(dataset[0])), int(.75 * len(dataset[0])), replace=False)
priors = np.zeros(len(dataset[0]))
priors[random_idx] = 1
tq_priors = np.diag(priors)
none_priors = np.zeros((len(dataset[0]), len(dataset[0])))

In [23]:
jm = JAMIE(**kwargs, debug=True, enable_memory_logging=True)
jm_data = jm.fit_transform(dataset=dataset)

use random seed: 666
Shape of Raw data
Dataset 0: (8981, 34104)
Dataset 1: (8981, 19836)
Device: cpu
---------------------------------
Find correspondence between Dataset 1 and Dataset 2
epoch:[500/2000] err:1774.8389 alpha:23.8913
epoch:[1000/2000] err:324.3965 alpha:0.0213
epoch:[1500/2000] err:152.1128 alpha:0.0015
epoch:[2000/2000] err:67.9232 alpha:0.0002
Finished Matching!
---------------------------------
Train coupled autoencoders
Epoch: 100 - KL: 0.0026  Rec: 1.7848  CosSim: 0.0364  F: 0.0016
Epoch: 200 - KL: 0.0050  Rec: 1.3758  CosSim: 0.0223  F: 0.0013
Epoch: 300 - KL: 0.0063  Rec: 1.6501  CosSim: 0.0245  F: 0.0011
Epoch: 400 - KL: 0.0101  Rec: 1.3823  CosSim: 0.0135  F: 0.0006
Epoch: 500 - KL: 0.0127  Rec: 1.6549  CosSim: 0.0118  F: 0.0005
epoch:[500/10000]: loss:1.522530
Epoch: 600 - KL: 0.0182  Rec: 1.3020  CosSim: 0.0128  F: 0.0006
Epoch: 700 - KL: 0.0282  Rec: 1.5264  CosSim: 0.0158  F: 0.0008
Epoch: 800 - KL: 0.0357  Rec: 1.2886  CosSim: 0.0141  F: 0.0006
Epoch: 900 -

# scGLUE

In [5]:
np.random.seed(42)

In [6]:
import scanpy as sc

dataset_name = 'scGLUE'
dataset_color = 'yellow'
modality_names = ['RNA', 'ATAC']
data_folder = '../data/scGLUE/'

data1 = sc.read_h5ad(data_folder + 'Chen-2019-RNA.h5ad')
type1 = data1.obs.cell_type.to_numpy()
fname1 = data1.var.name.to_numpy()
data1 = data1.X.todense()

data2 = sc.read_h5ad(data_folder + 'Chen-2019-ATAC.h5ad')
type2 = data2.obs.cell_type.to_numpy()
fname2 = data2.var.index.to_numpy()
data2 = data2.X.todense()

# Labels
labels = [type1, type2]
features = [fname1, fname2]

In [7]:
# Preprocessing
data1 = preprocessing.scale(data1, axis=0)
data2 = preprocessing.scale(data2, axis=0)
data1[np.isnan(data1)] = 0  # Replace NaN with average
data2[np.isnan(data2)] = 0
# data1 = preprocessing.MinMaxScaler().fit_transform(data1)
# data2 = preprocessing.MinMaxScaler().fit_transform(data2)
dataset = [data1, data2]

# Replace NULL feature names
for i in range(len(features)):
    if features[i] is None:
        features[i] = np.array([f'Feature {i}' for i in range(dataset[i].shape[1])])
        
# Train-Test Imputation
train_size = int(.8 * len(data1))
train_idx = np.random.choice(range(len(data1)), train_size, replace=False)
test_idx = np.array(list(set(range(len(data1))) - set(train_idx)))

# Reduced Priors
full_priors = np.eye(len(dataset[0]))
random_idx = np.random.choice(range(len(dataset[0])), int(.5 * len(dataset[0])), replace=False)
priors = np.zeros(len(dataset[0]))
priors[random_idx] = 1
half_priors = np.diag(priors)
random_idx = np.random.choice(range(len(dataset[0])), int(.75 * len(dataset[0])), replace=False)
priors = np.zeros(len(dataset[0]))
priors[random_idx] = 1
tq_priors = np.diag(priors)
none_priors = np.zeros((len(dataset[0]), len(dataset[0])))

/home/thema/miniconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:585: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/home/thema/miniconda3/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:235: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
/home/thema/miniconda3/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:254: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(
/home/thema/miniconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:585: FutureWarning: np.matrix usage is deprecated 

In [8]:
jm = JAMIE(**kwargs, debug=True, enable_memory_logging=True)
jm_data = jm.fit_transform(dataset=dataset)

use random seed: 666
Shape of Raw data
Dataset 0: (9190, 28930)
Dataset 1: (9190, 241757)
Device: cpu
---------------------------------
Find correspondence between Dataset 1 and Dataset 2
epoch:[500/2000] err:31392.3984 alpha:3819.6143
epoch:[1000/2000] err:8624.8457 alpha:1050.6832
epoch:[1500/2000] err:2395.9426 alpha:291.6512
epoch:[2000/2000] err:198.7031 alpha:16.8099
Finished Matching!
---------------------------------
Train coupled autoencoders
Epoch: 100 - KL: 0.0031  Rec: 1.7631  CosSim: 0.0261  F: 0.0012
Epoch: 200 - KL: 0.0050  Rec: 2.0951  CosSim: 0.0218  F: 0.0010
Epoch: 300 - KL: 0.0056  Rec: 1.9862  CosSim: 0.0123  F: 0.0005
Epoch: 400 - KL: 0.0111  Rec: 1.8260  CosSim: 0.0171  F: 0.0007
Epoch: 500 - KL: 0.0132  Rec: 1.3674  CosSim: 0.0134  F: 0.0005
epoch:[500/10000]: loss:1.643861
Epoch: 600 - KL: 0.0148  Rec: 1.6133  CosSim: 0.0156  F: 0.0006
Epoch: 700 - KL: 0.0275  Rec: 1.7142  CosSim: 0.0167  F: 0.0006
Epoch: 800 - KL: 0.0358  Rec: 1.9653  CosSim: 0.0153  F: 0.0005